# Empowering Chatbots with Retrieval Augmented Generation (RAG)!

Imagine supercharging your chatbot with the ability to swiftly comb through the vast resources of the internet or specific databases, enabling it to provide more up-to-date and insightful answers. This is where the magic of RAG comes into play!

Here's the scoop: When a user poses a question to your chatbot, instead of solely relying on its internal knowledge, the chatbot sends the query to an API, such as an Internet Search or VectorDB. It then leverages the results to construct a more comprehensive and informed response. Think of it as your chatbot conducting quick online research before delivering an answer!

## What Makes RAG So Remarkable?

- Cost-Effective: Unlike the resource-intensive process of fine-tuning, RAG is more lightweight and easier to manage.
- Human Analogy: Consider your information needs. Would you attempt to absorb the entirety of a library's content, or would you locate the precise book and extract what you require? RAG operates on the latter principle. 

For businesses, instead of laboriously sifting through heaps of documents to train your chatbot, you can simply store them in a VectorDB. When a query arises, your chatbot can retrieve the pertinent information and present it in a user-friendly format.

## RAG vs. Fine-Tuning

- Choose Fine-Tuning when you need to refine your chatbot's behavior.
- Opt for RAG when you want your chatbot to tap into external knowledge bases. Bonus: RAG works seamlessly even with smaller 7B models!

Our journey begins with a beginner's example, addresses challenges, and delves into advanced use cases in the world of RAG. Let's get started!

In [ ]:
%pip install openai
%pip install requests
%pip install duckduckgo_search

In [ ]:
# Import the required modules
from openai import OpenAI
import IPython


# Set the client and api key

llm_client = OpenAI(
    api_key=''
)


# Let's set our model
MODEL = "gpt-3.5-turbo"


As evident, when utilizing zero-shot learning, the responses generated are rooted in the LLM's training data, which is limited to information available up to its training cutoff date in September 2021.

In [ ]:
# Zero-Shot

question = 'Who is the CEO of Twitter in 2023?'

response = llm_client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": question}
    ],
    temperature=0,
)

IPython.display.Markdown(response.choices[0].message.content)




Now, let's introduce a function that enables us to leverage DuckDuckGo and adapt the prompt in a way that allows us to present both the query and the search results concurrently...

In [ ]:
# RAG - DuckDuckGo
from duckduckgo_search import DDGS
from gptrim import trim

def search_internet(query):
    """
    Use Duckduck go Search
    """
    
    count = 5
    result_text=''
    with DDGS() as ddgs:
        search_results = [r for r in ddgs.text(query, max_results=count)]
        for result in search_results:
            title = result.get('title', '')
            snippet = result.get('body', '')
            url = result.get('href', '')
            result_text += f'Title: {title}\nSnippet: {snippet}\nURL: {url}\n\n'

        search_prompt = f"""
        Based on the internet search results provided in <>, provide an answer to the query [] if it is relevant along with a source URL. \
        If there are no internet search results or if they are not relevant then say \"Please try again.\"\
        
        context:<{trim(result_text)}>
        query:[{query}]
        """

        print ('Original text source:\n'+result_text)

        print ('Trimmed text source:\n'+trim(result_text))

        return {"role":"system","content": search_prompt}



With our function at our disposal, let's approach the task by first sending our query to the DuckDuckGo function and then supplying the adapted prompt to the LLM.

In [ ]:

# STEP 1 - ASK THE QUESTION
question = 'Who is the CEO of Twitter in 2023?'


# STEP 2 - SUBMIT THE QUESTION TO AN API TO GET RESULTS AND CREATE A NEW PROMPT FOR THE LLM

prompt = search_internet(question)


# STEP 4 - SEND THE MODIFIED PROMPT TO THE LLM 

response = llm_client.chat.completions.create(
    model=MODEL,
    messages=[
        prompt
    ],
    temperature=0,
)


IPython.display.Markdown(response.choices[0].message.content)

Now, let's delve into the challenges at hand:

### Challenge 1 - Simplifying Complex Queries
One prominent issue is the substantial text required to extract a concise answer. This not only impacts comprehension but also results in the usage of more tokens, potentially incurring additional costs. However, you can employ modules like "gptrim" to condense the text by approximately 50%, eliminating spaces, stop words, and the like. Remarkably, the LLM, particularly GPT-3 and 4, remains adept at understanding and responding effectively to such condensed input.

### Challenge 2 - Transforming User Queries into Search Queries
Accepting the user's query as-is doesn't always lead to optimal results. To address this, we need a mechanism to modify the query into an effective search query. Here, the LLM can assist in generating appropriate search queries using the function calling API. While several frameworks like Langchain and Semantic Kernel are available, I have found that function calling offers a straightforward and practical approach.

### Challenge 3 - Efficient Decision-Making
Consider that this setup operates within a chatbot application, across platforms such as websites, Microsoft Teams, Slack, Alexa, or robotic interfaces. In such scenarios, we don't want to trigger internet searches unnecessarily. To optimize this, we can introduce functions with distinct names and descriptions to the LLM's function calling API. The LLM then possesses the discretion to determine whether a function call is warranted, allowing for context-appropriate responses.

### Challenge 4 - Accessing Company Documents
Intriguingly, there's a desire to engage with company documents effectively. While you can explore this avenue with Confluence, Stack Overflow, and Sharepoint APIs, it's important to note that these solutions rely on keyword-based searches. This necessitates the exact matching of keywords found in document titles for retrieval. Alternatively, leveraging a vector database introduces the capacity for semantic searches. Here, you can import all your documents or text snippets and harness the power of semantic search by querying the concept or idea most closely aligned with the content you seek. We'll delve into a practical example of this in a separate notebook.

# Retrieval Augmented Generation with Function Calling

In this approach, we entrust the LLM with the authority to discern when a function (often referred to as "skills") is necessary, generate the appropriate search query, and then incorporate the results into the LLM to obtain the final answer.

Here's the breakdown of this process:

1. **Function Development:** We create functions that act as additional "eyes" (read) and "hands" (actions/write) for the LLM.

2. **Function Definitions:** We define these functions in a list, specifying the exact function name and a description. The LLM utilizes this information to select the relevant function. We also define the properties required for each function.

3. **Query Submission:** Your query is submitted to the LLM, along with the function definitions.

4. **Function Requirement:** If the LLM determines that a function is required, you will receive a response that includes the necessary function and its associated properties.

5. **Function Execution:** You utilize the information provided by the LLM to internally trigger and execute the required function. This could involve searching the internet, performing specific actions, or any other defined tasks.

6. **Final Answer:** After executing the function, you return to the LLM with the required information, and the LLM generates the final response.

This approach streamlines the interaction between the LLM and the external functions, enhancing the model's ability to deliver precise and context-aware responses.

In [ ]:
# Make sure to run the search_internet function before this!
import json

## Requires a function 
QUESTION = "Who is Twitter's CEO in 2023?"
## Doesn't require a function
#QUESTION = "How can i make a cup of tea?"


# Let us define the search_internet function for the LLM...
skill_definitions = [
      {
        "name": "search_internet",
        "description": "Searches the internet to provide context to a query if required",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The query to search for"
                }
            },
            "required": ["query"]
        }
    }
]


messages = [
    {"role": "system", "content": "You have tools that add more up to date context to queries should they be required, otherwise act as a helpful assistant."},
    {"role": "user", "content": QUESTION}
    ]

response = llm_client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=skill_definitions, # we present our skill definitions to the LLM. Think of it as a toolbox of functions :)
    function_call='auto' # auto means that we let the LLM decide when to use a function. You can also set a static function name here so that it will always call it.
)

print (f'Look out for the function_call and finish_reason keys shown in the JSON below:\n {response}') 


### Let us take the JSON response and load it


### Do we need a skill?
if 'function_call' in response.choices[0].finish_reason:

    # tie response with our functions
    available_functions = {
        'search_internet': search_internet
    }


    # extract function name and arguments from the LLM
    function_name = response.choices[0].message.function_call.name
    function_args = json.loads(response.choices[0].message.function_call.arguments)

    function2call = available_functions.get(function_name)

    # call it appropriately
    function_response = function2call(**function_args)

    # append the response to the message history
    messages.append(function_response)

    # send the message history to the LLM again without functions.
    new_response = llm_client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    #functions=skill_definitions,
    #function_call='auto' 
)
    
    # Result

    print ('\n\n'+str(new_response.choices[0].message.content))
    



else:
    print(response.choices[0].message.content) 





As evident, the LLM has determined the necessity of a tool and has furnished the essential arguments for its execution. Notably, the query was also appropriately modified to align with this requirement.

While this approach is effective, it's crucial to be vigilant, as the LLM may occasionally propose function names that do not exist. Implementing a conditional check to ensure that the returned function name aligns with the predefined ones is advisable to maintain control and accuracy.

With this approach, you have the flexibility to integrate as many "hands" and "eyes" as necessary into your chatbot, enabling it to adapt and extend its capabilities as needed.